In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests, re, collections

In [9]:
### Defined a finction to extract text
def get_text(t, *args, **kwargs):
    tag = t.find(*args, **kwargs)
    return tag.text if tag else None

In [16]:
#Web scraping job information from Seek and adding it to jobs list after every for loop
def get_jobs(url):
    jobs=[]
    while url:
        print(url)
        html_unparsed = requests.get(url)
        soup = BeautifulSoup(html_unparsed.text,'lxml')
        
        pattern = re.compile(".*")
        for j in soup.find_all("div", {"data-search-sol-meta": pattern}):
            job = collections.OrderedDict()
            try:
                job['job_title']= j.find('a',{'data-automation':'jobTitle'}).text
                job['location']=get_text(j, 'a',{'data-automation':'jobLocation'})
                job['salary'] = get_text(j, 'span',{'data-automation':'jobSalary'})
                job['company'] = get_text(j, 'a',{'data-automation':'jobCompany'})
                job['class'] = get_text(j, 'a',{'data-automation':'jobClassification'})
                
                ind_page_url = "https://www.seek.com.au" + j.find('a',{'data-automation':'jobTitle'})['href']
                html_unparsed1 = requests.get(ind_page_url)
                ind_soup = BeautifulSoup(html_unparsed1.text,'lxml')
                
                job['job_type'] = get_text(ind_soup,'dd',{'data-automation':'job-detail-work-type'})
                job['description'] = get_text(ind_soup,'div',{'class':'templatetext'})
                if not job['description']:
                    job['description'] = get_text(ind_soup,'div',{'data-automation':'mobileTemplate'})
                
                jobs.append(job)
            except:
                AttributeError
        ## Next page link href
        next_link = soup.find("a", {"data-automation": "page-next"}, string="")
        if next_link != None:
            url = "https://www.seek.com.au" + next_link['href']
        else:
            url = None

    return jobs

In [17]:
urls1= [
    'https://www.seek.com.au/jobs/in-All-Australia?daterange=31&keywords=%22machine%20learning%22%20or%20%22data%20scientist%22%20or%20%22data%20science%22%20or%20%22data%20analysts%22%20or%20%22data%20science%20specialists%22',
    'https://www.seek.com.au/jobs/in-All-Australia?keywords=%22Data%20engineer%22&page=7&sortmode=KeywordRelevance',
    'https://www.seek.com.au/jobs/in-All-Australia?keywords=%22Business%20intelligence%22&sortmode=KeywordRelevance'
]

jobs1 = []
for url in urls1:
    jobs1.extend(get_jobs(url))

https://www.seek.com.au/jobs/in-All-Australia?daterange=31&keywords=%22machine%20learning%22%20or%20%22data%20scientist%22%20or%20%22data%20science%22%20or%20%22data%20analysts%22%20or%20%22data%20science%20specialists%22
https://www.seek.com.au/jobs/in-All-Australia?keywords=%22machine+learning%22+or+%22data+scientist%22+or+%22data+science%22+or+%22data+analysts%22+or+%22data+science+specialists%22&daterange=31&page=2
https://www.seek.com.au/jobs/in-All-Australia?keywords=%22machine+learning%22+or+%22data+scientist%22+or+%22data+science%22+or+%22data+analysts%22+or+%22data+science+specialists%22&daterange=31&page=3
https://www.seek.com.au/jobs/in-All-Australia?keywords=%22machine+learning%22+or+%22data+scientist%22+or+%22data+science%22+or+%22data+analysts%22+or+%22data+science+specialists%22&daterange=31&page=4
https://www.seek.com.au/jobs/in-All-Australia?keywords=%22machine+learning%22+or+%22data+scientist%22+or+%22data+science%22+or+%22data+analysts%22+or+%22data+science+specialis

https://www.seek.com.au/jobs/in-All-Australia?keywords=%22Business+intelligence%22&sortmode=KeywordRelevance&page=14
https://www.seek.com.au/jobs/in-All-Australia?keywords=%22Business+intelligence%22&sortmode=KeywordRelevance&page=15
https://www.seek.com.au/jobs/in-All-Australia?keywords=%22Business+intelligence%22&sortmode=KeywordRelevance&page=16
https://www.seek.com.au/jobs/in-All-Australia?keywords=%22Business+intelligence%22&sortmode=KeywordRelevance&page=17
https://www.seek.com.au/jobs/in-All-Australia?keywords=%22Business+intelligence%22&sortmode=KeywordRelevance&page=18
https://www.seek.com.au/jobs/in-All-Australia?keywords=%22Business+intelligence%22&sortmode=KeywordRelevance&page=19


In [18]:
#Converting list into a dataframe
df_jobs= pd.DataFrame(jobs1)

In [19]:
df_jobs.shape

(1032, 7)

In [20]:
df_jobs.head()

,job_title,location,salary,company,class,job_type,description
0,Data Scientist | Smart Cities,Sydney,$900 - $1200 p.d.,Talenza,Science & Technology,Contract/Temp,\nTalenza has been engaged by an Icon Australi...
1,Data Scientist l Machine Learning Engineer l $...,Sydney,"$130,000 - $179,999",None,Information & Communication Technology,Full Time,Great team environment/ office space Perman...
2,Data Scientist,Melbourne,Competitive,FourQuarters Recruitment,Information & Communication Technology,Full Time,Data Scientist | Melbourne CBD | Permanent\n\n...
3,Lead Data Scientist | Python | Machine Learning,Sydney,$1250 - $1600 p.d.,Talenza,Science & Technology,Contract/Temp,\nTalenza has been engaged by industry leading...
4,Data Scientist,Sydney,None,Greythorn Experis,Information & Communication Technology,Contract/Temp,\n\n\nHands-on experience with common economet...
5,Data Scientist | Customer Analytics | Fintech,Sydney,$160-170 K Plus Superannuation,HCM Australia,Information & Communication Technology,Full Time,\n \n \nData Scientist | Customer Analytics | ...
6,Senior Data Scientist / Data Products Consult...,Melbourne,up to $1200pd,Correlate Resources,Information & Communication Technology,Contract/Temp,In this exciting contract opportunity we are l...
7,Data Scientist,Sydney,None,Motion Recruitment,Information & Communication Technology,Full Time,About the business and the role My client is ...
8,Data Science and Analytics Specialist (Data Sc...,"Newcastle, Maitland & Hunter",None,GP Synergy,Information & Communication Technology,Full Time,\n\nFull-time position\nNewcastle office locat...
9,Data Scientist,Sydney,None,Foxtel Management Pty Limited,Information & Communication Technology,Full Time,"At Foxtel, we’re bringing television to Austra..."


In [ ]:
### saving file as csv to use it for the next step, index set to False 

In [22]:
df_jobs.to_csv('Final_to_be_cleaned.csv', index=False)

In [15]:
df.to_csv('Job_search.csv')